## Downloads

In [3]:
%%capture
!pip install preprocessor
!pip install vaderSentiment
!pip install torchmetrics
!pip install sentence-transformers

## Imports

In [215]:
import os
import re
import json
import pickle
from datetime import date

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import preprocessor as p
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vincentdandenault/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincentdandenault/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Run Flags and File Paths

In [140]:
date_today = date.today()

random_seed = 42
target_names = ['Not Censord', 'Censord']

RUN_PREPROCESSING = False
FIT_CORPUS_FEATURE_SPACE = True

FEATURE_SPACE = 'SentenceEmbeddings'

LANGUAGE_RUN = 'English'

csv_path = 'Output/df_cen.csv'
data_path = 'Data'
results_path = 'Results'
vector_path = 'vectors'
clean_dataframe_path = 'Output/df_clean.csv'

df_input = pd.read_csv(csv_path)

/var/folders/_m/v82wb7j964v3b7b70x4rrvvr0000gn/T/ipykernel_60908/1187662931.py:19: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_input = pd.read_csv(csv_path)


In [141]:
df_input.head(10)

,Unnamed: 0,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,filter_level,lang,timestamp_ms,linked,display_text_range,withheld_in_countries,extended_entities,possibly_sensitive,retweeted_status,withheld_copyright
0,1,2021-02-26 10:48:35+00:00,1365252415444946945,1365252415444946944,#Balakot \nPak Army is our pride ❤️👍 https://t...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,en,2021-02-26 10:48:35.662,no,"[0, 35]",['IN'],"{'media': [{'id': 1365252409015033857, 'id_str...",0.0,NaN,NaN
1,2,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,no,NaN,['IN'],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
2,3,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,retweeted,NaN,['IN'],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
3,4,2021-02-26 10:38:57+00:00,1365249991137251328,1365249991137251328,RT @SaniaNishtar: سید ابراہیم کا تعلق قبائلی ض...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,ur,2021-02-26 10:38:57.662,no,NaN,['IN'],NaN,NaN,{'created_at': 'Fri Feb 26 05:23:00 +0000 2021...,NaN
4,5,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,no,NaN,['IL'],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
5,6,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,retweeted,NaN,['IL'],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
6,7,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,no,NaN,['IN'],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
7,8,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,retweeted,NaN,['IN'],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
8,9,2021-02-26 10:59:24+00:00,1365255137552457730,1365255137552457728,RT @iVeenaKhan: (اے پیغمبرﷺ! لوگوں سے) کہہ دو ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,ur,2021-02-26 10:59:24.663,no,NaN,['IN'],NaN,NaN,{'created_at': 'Fri Feb 26 05:42:25 +0000 2021...,NaN
9,10,2021-02-26 10:59:24+00:00,1365255137552457730,1365255137552457728,RT @iVeenaKhan: (اے پیغمبرﷺ! لوگوں سے) کہہ دو ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,low,ur,2021-02-26 10:59:24.663,retweeted,NaN,['IN'],NaN,NaN,{'created_at': 'Fri Feb 26 05:42:25 +0000 2021...,NaN


 ## Preprocessing

In [142]:
list_of_countries = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', \
                   'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', \
                   'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', \
                   'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', \
                   'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', \
                   'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', \
                   'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', \
                   'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', \
                   'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', \
                   'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', \
                   'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", \
                   'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark',
                   'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', \
                   'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', \
                   'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', \
                   'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', \
                   'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', \
                   'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', \
                   'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', \
                   'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', \
                   'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', \
                   'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", \
                   'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", \
                   'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', \
                   'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', \
                   'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', \
                   'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', \
                   'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', \
                   'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', \
                   'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', \
                   'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', \
                   'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', \
                   'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', \
                   'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', \
                   'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia',\
                   'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', \
                   'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', \
                   'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', \
                   'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', \
                   'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', \
                   'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', \
                   'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', \
                   'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', \
                   'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', \
                   'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', \
                   'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', \
                   'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', \
                   'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', \
                   'Zambia', 'Zimbabwe']

def find_country(x):
        for country in list_of_countries:
            if x and country in x:
                return country
        return None
    
def normalize(array):
        return (array - np.min(array)) / (np.max(array) - np.min(array))
    
def preprocess_df(df_input):
    features_worth_keeping = ["text", "truncated", "user",
                    "withheld_in_countries", "entities", "lang",
                    "possibly_sensitive", "extended_tweet"]
    
    df_cen = df_input[features_worth_keeping]
    df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)
    dfRaw = df_cen.values
    for line in dfRaw:
        #if not pd.isna(line[-1]):
            #line[0] = line[-1]["full_text"]
            
        #remove urls from tweets
        #they are shortened anyway so we can't make use of them
        line[0] = re.sub(r'http\S+', '', str(line[0]))
        
        #flatten retweets
        line[0] = re.sub(r'RT @\S+:', '', str(line[0]))
    
    dfRaw = np.delete(dfRaw, len(features_worth_keeping)-1, axis=1) #remove "extended_tweet"
    features_worth_keeping.remove("extended_tweet")
    
    dfRaw = np.delete(dfRaw, 1, axis=1) #remove "truncated"
    features_worth_keeping.remove("truncated")
    
    for line in dfRaw:
        line[3] = [x["text"] for x in line[3]["hashtags"]]
    features_worth_keeping[3] = "hashtags"
    
    verified = [line[1]["verified"] for line in dfRaw]
    followers = [line[1]["followers_count"] for line in dfRaw]
    user_id = [line[1]["id"] for line in dfRaw]
    
    location = [find_country(line[1]["location"]) for line in dfRaw]
    
    df_raw = np.c_[df_raw, verified, followers, location, user_id]
    features_worth_keeping += ["verified_account", "followers_count", "location", "user_id"]
    withheld = []
    for line in df_raw:
        if not isinstance(line[2], list):
            line[2] = []
        withheld.append(len(line[2]) != 0)
            
    df_raw = np.c_[df_raw, withheld]
    features_worth_keeping += ["withheld_anywhere"]
    
    sentiment = SentimentIntensityAnalyzer() #we made the assumption that sentiment analysis for this analyzer only works for english
    res = np.array([[x for x in sentiment.polarity_scores(line[0]).values()] if line[4] == "en" 
                    else [0.0, 0.0, 0.0, 0.0] for line in df_raw])

    df_raw = np.c_[df_raw, res]
    features_worth_keeping += ["neg", "neu", "pos", "compound"]
    
    followers_count = np.array([line[1]["followers_count"] for line in df_raw])
    favourites_count = np.array([line[1]["favourites_count"] for line in df_raw])
    statuses_count = np.array([line[1]["statuses_count"] for line in df_raw])
    
    score = (1/3) * (normalize(followers_count) + normalize(favourites_count) + normalize(statuses_count))
    df_raw = np.c_[df_raw, score]
    worthKeeping += ["popularity_score"]
    
    
    return pd.DataFrame(df_raw, columns = worthKeeping)

In [240]:
if RUN_PREPROCESSING:
    df = preprocess_df(df_input)
else: 
    df = pd.read_csv(clean_dataframe_path)

## Splitting the Data by Language

In [144]:
def make_country_dataframes(df): 
    dataframes_dict = {}
    
    df_english = df[df['lang'] == "en"] 
    dataframes_dict['English'] = df_english
    
    df_turkish = df[df['lang'] == "tr"] 
    dataframes_dict['Turkish'] = df_turkish
    
    df_urdu = df[df['lang'] == "ur"]
    dataframes_dict['Urdu'] = df_urdu
    
    df_japanese = df[df['lang'] == "ja"] 
    dataframes_dict['Japanese'] = df_japanese
    
    df_spanish = df[df['lang'] == "es"] 
    dataframes_dict['Spanish'] = df_spanish
    
    df_thai = df[df['lang'] == "th"] 
    dataframes_dict['Thai'] = df_thai
    
    df_portuguese = df[df['lang'] == "pt"] 
    dataframes_dict['Portuguese'] = df_portuguese
    
    df_arabic = df[df['lang'] == "ar"] 
    dataframes_dict['Arabic'] = df_arabic
    
    df_indian = df[df['lang'] == "in"] 
    dataframes_dict['Indian'] = df_indian
    
    return dataframes_dict

In [145]:
dataframes_dict = make_country_dataframes(df)

## Feature Encoding

In [146]:
features_encoded = ['possibly_sensitive', 'verified_account',\
                    'followers_count', 'user_id', 'neg', 'neu', \
                    'pos', 'compound', 'popularity_score']

In [213]:
def encode_features(df, PCA_flag=True):
    df_features = df.copy()
    if FIT_CORPUS_FEATURE_SPACE:
        corpus = [sentence if isinstance(sentence, str) else '' for sentence in list(df_features['text'].values)]
        
        if FEATURE_SPACE == 'BOW':
            bow_texts = CountVectorizer().fit_transform(corpus)
            with open(('bow_vectors_' + str(LANGUAGE_RUN) + '.pickle'), 'wb') as pkl:
                pickle.dump(bow_texts, pkl)
            text_vector = bow_texts
        
        elif FEATURE_SPACE == 'TFIDF':
            tfidf_vectors = TfidfVectorizer().fit_transform(corpus) 
            with open(('tfidf_vectors_' + str(LANGUAGE_RUN) + '.pickle'), 'wb') as pkl:
                pickle.dump(tfidf_vectors, pkl)
            text_vector = tfidf_vectors
        
        else: 
            model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
            tweets_embeddings = model.encode(sentences=corpus, batch_size=32, 
                                             show_progress_bar=True, convert_to_numpy=True, 
                                            normalize_embeddings=True)
            with open(('tweets_embeddings_' + str(LANGUAGE_RUN) + '.pickle'), 'wb') as pkl:
                pickle.dump(tweets_embeddings, pkl)
            text_vector = tweets_embeddings
        
    else: 
        if FEATURE_SPACE == 'BOW':
            with open(os.path.join(vector_path,'bow_vectors.pickle'), 'rb') as pkl:
                text_vector = pickle.load(pkl)
        elif FEATURE_SPACE == 'TFIDF': 
            with open(os.path.join(vector_path,'tfidf_vectors.pickle'), 'rb') as pkl:
                text_vector = pickle.load(pkl)
        else: 
            with open(os.path.join(vector_path,'tweets_embeddings.pickle'), 'rb') as pkl:
                text_vector = pickle.load(pkl) 
    
    country_label = preprocessing.LabelEncoder()
    countries_encoded = country_label.fit_transform(list(df.location.values))
    df_features['Country_encoded'] = countries_encoded
    
    df_features = df_features.astype({"possibly_sensitive": float, "verified_account": float,\
                                'followers_count':int, 'user_id': int, 'neg': float, 'neu': float, 
                               'pos': float, 'compound': float, 'popularity_score': float})
    
    y = df_features["withheld_anywhere"].astype(int)
    
    X = df_features[features_encoded].copy().to_numpy()
    #X = np.concatenate((X, np.array(text_vector)), axis=1)
    
    X = text_vector
    
    if PCA_flag: 
         X = PCA(n_components=0.95, svd_solver='full').fit_transform(X)
    
    return X, y
    
    

In [148]:
df = dataframes_dict[LANGUAGE_RUN]
X, y = encode_features(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Batches:   0%|          | 0/808 [00:00<?, ?it/s]

In [149]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20664, 320)
(5166, 320)
(20664,)
(5166,)


## Training the Models

In [150]:
base_model = LogisticRegression(random_state=random_seed)
y_pred = base_model.fit(X_train, y_train).predict(X_test)
res = classification_report(y_test, y_pred, target_names=target_names, output_dict=True) 
print("Baseline model - Logisitic Regression: ")
print(res)

Baseline model - Logisitic Regression: 
{'Not Censord': {'precision': 0.7285992217898832, 'recall': 0.7296639064783244, 'f1-score': 0.7291311754684837, 'support': 2053}, 'Censord': {'precision': 0.8215434083601286, 'recall': 0.8207516864760681, 'f1-score': 0.8211473565804275, 'support': 3113}, 'accuracy': 0.7845528455284553, 'macro avg': {'precision': 0.7750713150750059, 'recall': 0.7752077964771962, 'f1-score': 0.7751392660244556, 'support': 5166}, 'weighted avg': {'precision': 0.7846068200851162, 'recall': 0.7845528455284553, 'f1-score': 0.7845795633510778, 'support': 5166}}


In [206]:
models = {'SVM': SVC(random_state=random_seed), 
          'Random Forest': RandomForestClassifier(random_state=random_seed), 
          'Gaussian Naive Bayes': GaussianNB()}

def run_models(models, X_train, X_test, y_train, y_test, save_results=True): 
    scores = {}
    for name in models.keys(): 
        y_pred = models[name].fit(X_train, y_train).predict(X_test)
        res = classification_report(y_test, y_pred, target_names=target_names, output_dict=True, zero_division=0) 
        scores[name] = res
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names, output_dict=False, zero_division=0))
    if save_results: 
        with open(os.path.join(results_path, ('results_' + str(date_today) + '.txt')), 'w') as f:
            for key, value in scores.items(): 
                f.write('%s:%s\n' % (key, value))
                f.write('\n')

## Multiple Country Runner

In [204]:
def make_dfs_by_country(df, country_list):
    df_list = []
    for country in country_list: 
        df_tmp = df[df['location'] == country].copy()
        df_list.append(df_tmp)
    return df_list

In [214]:
model_to_run = {'SVM': SVC(random_state=random_seed)}
df_countries = df.copy()
countries = ['France', 'Turkey', 'Germany', 'India']
df_list = make_dfs_by_country(df, countries)  
for idx, df in enumerate(df_list): 
    X, y = encode_features(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Country: ' + str(countries[idx]))
    run_models(model_to_run, X_train, X_test, y_train, y_test)

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

Country: France
SVM
              precision    recall  f1-score   support

 Not Censord       0.89      0.39      0.54        44
     Censord       0.80      0.98      0.88       111

    accuracy                           0.81       155
   macro avg       0.85      0.68      0.71       155
weighted avg       0.83      0.81      0.79       155



Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Country: Turkey
SVM
              precision    recall  f1-score   support

 Not Censord       0.00      0.00      0.00         3
     Censord       0.77      1.00      0.87        10

    accuracy                           0.77        13
   macro avg       0.38      0.50      0.43        13
weighted avg       0.59      0.77      0.67        13



Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Country: Germany
SVM
              precision    recall  f1-score   support

 Not Censord       0.00      0.00      0.00         4
     Censord       0.93      1.00      0.96        55

    accuracy                           0.93        59
   macro avg       0.47      0.50      0.48        59
weighted avg       0.87      0.93      0.90        59



Batches:   0%|          | 0/18 [00:00<?, ?it/s]

Country: India
SVM
              precision    recall  f1-score   support

 Not Censord       0.72      0.96      0.82        67
     Censord       0.86      0.42      0.56        43

    accuracy                           0.75       110
   macro avg       0.79      0.69      0.69       110
weighted avg       0.77      0.75      0.72       110



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Country: China
SVM
              precision    recall  f1-score   support

 Not Censord       0.33      1.00      0.50         1
     Censord       1.00      0.33      0.50         3

    accuracy                           0.50         4
   macro avg       0.67      0.67      0.50         4
weighted avg       0.83      0.50      0.50         4



## Vizualisation

In [239]:
df.head(10)

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
7326,FRANCE: Sudanese Muslim illegal alien stabs im...,['FR'],[],en,0.0,False,2932,China,1116313997463183363,True,0.533,0.467,0.000,-0.9287,0.004599
7327,FRANCE: Sudanese Muslim illegal alien stabs im...,['FR'],[],en,0.0,False,2932,China,1116313997463183363,True,0.533,0.467,0.000,-0.9287,0.004599
21040,I am not fabricating these news....!,['IN'],[],en,0.0,False,5975,China,3311813269,True,0.000,1.000,0.000,0.0000,0.010005
21041,I am not fabricating these news....!,['IN'],[],en,0.0,False,5975,China,3311813269,True,0.000,1.000,0.000,0.0000,0.010005
21048,Is he a virologist? A doctor? Specialist in pu...,['IN'],[],en,0.0,False,411,China,41757618,True,0.000,0.923,0.077,0.2120,0.001944
21049,Is he a virologist? A doctor? Specialist in pu...,['IN'],[],en,0.0,False,411,China,41757618,True,0.000,0.923,0.077,0.2120,0.001944
30735,NaN,[],[],ur,0.0,False,1219,China,146044067,False,0.000,0.000,0.000,0.0000,0.003569
31618,NaN,['IN'],[],ur,0.0,False,481,China,1172595522827444226,True,0.000,0.000,0.000,0.0000,0.004586
31619,NaN,['IN'],[],ur,0.0,False,481,China,1172595522827444226,True,0.000,0.000,0.000,0.0000,0.004586
45301,full offense jimin is prettier than you,[],[],en,0.0,False,67,China,1109423371433852928,False,0.198,0.495,0.307,0.2732,0.002774


In [243]:
df_France = df[df['location'] == 'France'].copy()
France_counts = list(df_France.withheld_anywhere.value_counts().values)

df_Turkey = df[df['location'] == 'India'].copy()
Turkey_counts = list(df_Turkey.withheld_anywhere.value_counts().values)

df_Germany = df[df['location'] == 'Germany'].copy()
Germany_counts = list(df_Germany.withheld_anywhere.value_counts().values)

df_India = df[df['location'] == 'India'].copy()
India_counts = list(df_India.withheld_anywhere.value_counts().values)


In [246]:
labels = ['Not Censored', 'Censored']
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(2, 2, specs=specs,
                    subplot_titles=countries)

fig.add_trace(go.Pie(labels=labels, values=France_counts, scalegroup='one',
                     name="France"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=Turkey_counts, scalegroup='one',
                     name="Turkey"), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=Germany_counts, scalegroup='one',
                     name="Germany"), 2, 1)
fig.add_trace(go.Pie(labels=labels, values=India_counts, scalegroup='one',
                     name="India"), 2, 2)


fig.update_layout(title_text='Percentage of Censorded Tweets by Countries')
fig.show()